# News Sentiment Analysis

In [4]:
! pip install pandas numpy matplotlib nltk tensorflow scikit-learn

  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached scikit_learn-1.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadat

In [5]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-01-02 23:59:25.080607: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-02 23:59:25.083612: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-02 23:59:25.089253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735851565.101175 3742761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735851565.104776 3742761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-02 23:59:25.119027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [7]:
amt_data = pd.read_csv("dataset/AMT_Clean.csv")
r_data = pd.read_csv("dataset/R_Clean.csv")

In [9]:
amt_data.head()

,Unnamed: 0,headline,entity,majority_label
0,0,Russia and Poland Feud Over Putin Remarks on W...,Putin,Negative
1,1,Trump Moves to Lift Visa Restrictions on Polis...,Trump,Neutral
2,2,Trump-Backed U.S.-British Trade Deal Faces Hur...,Trump,Neutral
3,3,Trump’s Arrival in London Brings Controversy b...,Trump,Negative
4,4,Trump Steers Clear of War Footing Toward Iran,Trump,Neutral


In [11]:
labels_map_amt = {
    "Negative": 2,
    "Neutral": 0,
    "Positive": 1
}

amt_data['tag'] = amt_data['majority_label'].map(labels_map_amt)

In [13]:
r_data.head(10)

,Unnamed: 0,headline,entity,majority_label
0,0,Boris Johnson Joins Trump in Redefining Conser...,Trump,neutr
1,1,Trump Praises Controversial Hungarian Leader,Trump,neg
2,2,Stung by Trump’s Criticisms of Russian Gas Dea...,Trump,neutr
3,3,What ‘America First’ Means Under Trump Is Comi...,Trump,neutr
4,4,Trump Set to Meet With Russia’s Top Diplomat T...,Trump,neutr
5,5,"Trump Clashes With Macron on NATO, Trade and I...",Trump,neutr
6,6,Donald Trump Won’t Slow Efforts to Fight Clima...,Trump,pos
7,7,Trump to Attend NATO Summit in London,Trump,neutr
8,8,Trump-Erdogan Rapport Tested by Tensions Over ...,Trump,neutr
9,9,Trump Says Erdogan Visit to U.S. Is On U.S. Of...,Trump,neutr


In [14]:
r_map = {
    "neg": 2,
    "neutr": 0,
    "pos": 1
}
r_data['tag'] = r_data['majority_label'].map(r_map)

In [16]:
# Extract the text and labels and merge to one dataframe
amt_data = amt_data[['headline', 'tag']]
r_data = r_data[['headline', 'tag']]
data = pd.concat([amt_data, r_data])

In [17]:
data.head()

,headline,tag
0,Russia and Poland Feud Over Putin Remarks on W...,2.0
1,Trump Moves to Lift Visa Restrictions on Polis...,0.0
2,Trump-Backed U.S.-British Trade Deal Faces Hur...,0.0
3,Trump’s Arrival in London Brings Controversy b...,2.0
4,Trump Steers Clear of War Footing Toward Iran,0.0


In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/abdulmunim/nltk_data...


True

In [ ]:
# Let's create some more synthetic data by translating the headlines to synonym
from nltk.corpus import wordnet
import random

def replace_synonyms(sentence, n=2):
    words = sentence.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        synonyms = wordnet.synsets(words[idx])
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            words[idx] = synonym
    return ' '.join(words)

# add the synthetic data to the dataset
synthetic_data = data.copy()
synthetic_data['headline'] = synthetic_data['headline'].apply(lambda x: replace_synonyms(x))
data = pd.concat([data, synthetic_data])


In [43]:
# ! pip install googletrans==4.0.0-rc1
# More synthetic data by  back translation
from googletrans import Translator

translator = Translator()

def back_translate(sentence, src='en', intermediate='fr'):
    translated = translator.translate(sentence, src=src, dest=intermediate).text
    back_translated = translator.translate(translated, src=intermediate, dest=src).text
    return back_translated

In [45]:
# statistics pof the data
data.dropna(inplace=True)
data['tag'] = data['tag'].astype(int)

In [46]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['headline'])
sequences = tokenizer.texts_to_sequences(data['headline'])
padded_sequences = pad_sequences(sequences, padding='post', maxlen=100)

In [47]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['tag'], test_size=0.2, random_state=42)

In [58]:
# Model definition
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=24, input_length=100),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [59]:
print(X_train.shape)
print(y_train.shape)
print(set(y_train))  # Should print {0, 1, 2}
print(set(y_test))   # Should print {0, 1, 2}


(3913, 100)
(3913,)
{0, 1, 2}
{0, 1, 2}


In [60]:

# Train the model
history = model.fit(X_train, y_train, epochs=20
                    , validation_data=(X_test, y_test), batch_size=32)

Epoch 1/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.4265 - loss: 1.0573 - val_accuracy: 0.5465 - val_loss: 1.0176
Epoch 2/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5949 - loss: 0.9441 - val_accuracy: 0.6374 - val_loss: 0.8161
Epoch 3/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7583 - loss: 0.6086 - val_accuracy: 0.7211 - val_loss: 0.6977
Epoch 4/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8622 - loss: 0.4110 - val_accuracy: 0.7579 - val_loss: 0.6380
Epoch 5/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8848 - loss: 0.3075 - val_accuracy: 0.7569 - val_loss: 0.6821
Epoch 6/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9019 - loss: 0.2634 - val_accuracy: 0.7875 - val_loss: 0.6525
Epoch 7/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9179 - loss: 0.2242 - val_accuracy: 0.7988 - val_loss: 0.6079
Epoch 8/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9239 - loss: 0.2204 - val_accu

In [64]:
model.export("models/news_sentiment_model")

INFO:tensorflow:Assets written to: models/news_sentiment_model/assets


INFO:tensorflow:Assets written to: models/news_sentiment_model/assets


Saved artifact at 'models/news_sentiment_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor_40')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  123274182346176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274170739600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274170742064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274170742416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274170865568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274363616976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274182347936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274170868912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274171044032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123274171054240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12327417

# 2. Quantize the Model for Mobile

In [69]:
converter = tf.lite.TFLiteConverter.from_saved_model("models/news_sentiment_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS    # Enable TensorFlow Select ops.
]
tflite_model = converter.convert()

# Save the TFLite model
with open("models/news_sentiment_model.tflite", "wb") as f:
    f.write(tflite_model)

W0000 00:00:1735855134.285630 3742761 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1735855134.285662 3742761 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-03 00:58:54.285831: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/news_sentiment_model
2025-01-03 00:58:54.287406: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-03 00:58:54.287428: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: models/news_sentiment_model
2025-01-03 00:58:54.300363: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-03 00:58:54.340985: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: models/news_sentiment_model
2025-01-03 00:58:54.363136: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 77309 microseconds.
2025-01-03 0